<a href="https://colab.research.google.com/github/teamgaon/recommendation_algorithms_know/blob/main/220107_ji_num_to_text_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 
import tensorflow as tf
import keras
from numpy import array
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
import math
import matplotlib as mpl
from IPython.core.display import display, HTML
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, f1_score
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, LSTM, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
import urllib.request
import json
import datetime
import os
from datetime import datetime
from tqdm import tqdm
import warnings
from glob import glob


In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [24]:
warnings.filterwarnings(action='ignore') 
train1 = pd.read_csv('/content/gdrive/MyDrive/data/know/train/KNOW_2017.csv', encoding='utf-8')
train2 = pd.read_csv('/content/gdrive/MyDrive/data/know/train/KNOW_2018.csv', encoding='utf-8')
train3 = pd.read_csv('/content/gdrive/MyDrive/data/know/train/KNOW_2019.csv', encoding='utf-8')
train4 = pd.read_csv('/content/gdrive/MyDrive/data/know/train/KNOW_2020.csv', encoding='utf-8')

test1 = pd.read_csv('/content/gdrive/MyDrive/data/know/test/KNOW_2017_test.csv', encoding='utf-8')
test2 = pd.read_csv('/content/gdrive/MyDrive/data/know/test/KNOW_2018_test.csv', encoding='utf-8')
test3 = pd.read_csv('/content/gdrive/MyDrive/data/know/test/KNOW_2019_test.csv', encoding='utf-8')
test4 = pd.read_csv('/content/gdrive/MyDrive/data/know/test/KNOW_2020_test.csv', encoding='utf-8')

submission = pd.read_csv('/content/gdrive/MyDrive/data/know/sample_submission.csv', encoding='utf-8')

train2 = train2[train2['idx'] != 18048]
train3 = train3[train3['idx'] != 19871]

In [25]:
train3['bq1']

0       15
1        8
2        1
3       15
4       19
        ..
8550    10
8551     9
8552     7
8553    19
8554    16
Name: bq1, Length: 8554, dtype: int64

In [26]:
def bq1_to_text(num:int):
  if num == 1:
    return '산업유형은 농업, 임업, 어업.'
  elif num == 2:
    return '산업유형은 광업.'
  elif num == 3:
    return '산업유형은 제조업.'
  elif num == 4:
    return '산업유형은 전기, 가스, 증기 및 공기조절 공급업.'
  elif num == 5:
    return '산업유형은 수도·하수·폐기물 처리, 원료재생업.'
  elif num == 6:
    return '산업유형은 건설업.'
  elif num == 7:
    return '산업유형은 도매 및 소매업.'
  elif num == 8:
    return '산업유형은 운수 및 창고업.'
  elif num == 9:
    return '산업유형은 숙박 및 음식점업.'
  elif num == 10:
    return '산업유형은 정보통신업.'
  elif num == 11:
    return '산업유형은 금융 및 보험업.'
  elif num == 12:
    return '산업유형은 부동산업.'
  elif num == 13:
    return '산업유형은 전문, 과학 및 기술 서비스업.'
  elif num == 14:
    return '산업유형은 사업시설 관리, 사업지원 및 임대 서비스업.'
  elif num == 15:
    return '산업유형은 공공행정, 국방 및 사회보장 행정.'
  elif num == 16:
    return '산업유형은 교육 서비스업.'
  elif num == 17:
    return '산업유형은 보건업 및 사회복지 서비스업.'
  elif num == 18:
    return '산업유형은 예술, 스포츠 및 여가관련 서비스업.'
  elif num == 19:
    return '산업유형은 협회 및 단체, 수리 및 기타 개인 서비스업.'
  elif num == 20:
    return '산업유형은 가구내 고용활동 및 달리 분류되지 않은 자가소비 생산활동.'
  elif num == 21:
    return '산업유형은 국제 및 외국기관.'

train3['bq1'] = train3['bq1'].map(bq1_to_text)

In [27]:
train3['bq1']

0             산업유형은 공공행정, 국방 및 사회보장 행정.
1                       산업유형은 운수 및 창고업.
2                     산업유형은 농업, 임업, 어업.
3             산업유형은 공공행정, 국방 및 사회보장 행정.
4       산업유형은 협회 및 단체, 수리 및 기타 개인 서비스업.
                     ...               
8550                       산업유형은 정보통신업.
8551                   산업유형은 숙박 및 음식점업.
8552                    산업유형은 도매 및 소매업.
8553    산업유형은 협회 및 단체, 수리 및 기타 개인 서비스업.
8554                     산업유형은 교육 서비스업.
Name: bq1, Length: 8554, dtype: object

In [28]:
def bq2_to_text(num:int):
  if num == 1:
    return '근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함).'
  elif num == 2:
    return '근무업체 유형은 정부기관 및 산하기관(공기업 포함).'
  elif num == 3:
    return '근무업체 유형은 (재단, 사단)법인단체.'
  elif num == 4:
    return '근무업체 유형은 특정회사나 사업체에 소속되어 있지 않음 (자영업 포함).'
  elif num == 5:
    return '근무업체 유형은 학교법인 혹은 학원.'
  elif num == 6:
    return '근무업체 유형은 기타.'

train3['bq2'] = train3['bq2'].map(bq2_to_text)

In [29]:
train3['bq2']

0                  근무업체 유형은 정부기관 및 산하기관(공기업 포함).
1              근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함).
2       근무업체 유형은 특정회사나 사업체에 소속되어 있지 않음 (자영업 포함).
3              근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함).
4       근무업체 유형은 특정회사나 사업체에 소속되어 있지 않음 (자영업 포함).
                          ...                   
8550                      근무업체 유형은 (재단, 사단)법인단체.
8551           근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함).
8552           근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함).
8553    근무업체 유형은 특정회사나 사업체에 소속되어 있지 않음 (자영업 포함).
8554               근무업체 유형은 정부기관 및 산하기관(공기업 포함).
Name: bq2, Length: 8554, dtype: object

In [30]:
def bq3_to_text(num:int):
  if num == 1:
    return '근무업체 규모는 1명 ~ 4명.'
  elif num == 2:
    return '근무업체 규모는 5명 ~ 49명.'
  elif num == 3:
    return '근무업체 규모는 50명 ~ 99명.'
  elif num == 4:
    return '근무업체 규모는 100명 ~299명.'
  elif num == 5:
    return '근무업체 규모는 300명 ~ 999명.'
  elif num == 6:
    return '근무업체 규모는 1,000명 이상.'

train3['bq3'] = train3['bq3'].map(bq3_to_text)


In [31]:
train3['bq3']

0       근무업체 규모는 100명 ~299명.
1        근무업체 규모는 50명 ~ 99명.
2          근무업체 규모는 1명 ~ 4명.
3         근무업체 규모는 5명 ~ 49명.
4          근무업체 규모는 1명 ~ 4명.
                ...         
8550    근무업체 규모는 100명 ~299명.
8551       근무업체 규모는 1명 ~ 4명.
8552       근무업체 규모는 1명 ~ 4명.
8553      근무업체 규모는 5명 ~ 49명.
8554    근무업체 규모는 100명 ~299명.
Name: bq3, Length: 8554, dtype: object

In [32]:
def bq5_1_to_text(num:int):
  try :
    num = int(num)
    if num == 1:
      return '훈련기간은 1개월 미만.'
    elif num == 2:
      return '훈련기간은 1개월 이상~3개월 미만.'
    elif num == 3:
      return '훈련기간은 3개월 이상~6개월 미만.'
    elif num == 4:
      return '훈련기간은 6개월 이상~1년 미만.'
    elif num == 5:
      return '훈련기간은 1년 이상~2년 미만.'
    elif num == 6:
      return '훈련기간은 2년 이상.'
  except :
    return '훈련기간은 필요없다.'

train3['bq5_1'] = train3['bq5_1'].map(bq5_1_to_text)


In [33]:
train3['bq5_1']

0         훈련기간은 1년 이상~2년 미만.
1                훈련기간은 필요없다.
2                훈련기간은 필요없다.
3              훈련기간은 1개월 미만.
4                훈련기간은 필요없다.
                ...         
8550           훈련기간은 1개월 미만.
8551    훈련기간은 3개월 이상~6개월 미만.
8552             훈련기간은 필요없다.
8553             훈련기간은 필요없다.
8554             훈련기간은 필요없다.
Name: bq5_1, Length: 8554, dtype: object

## 2020년

In [34]:
train4['bq1'] = train4['bq1'].map(bq1_to_text)

In [35]:
train4['bq1']

0                  산업유형은 교육 서비스업.
1       산업유형은 공공행정, 국방 및 사회보장 행정.
2                      산업유형은 제조업.
3                  산업유형은 교육 서비스업.
4                      산업유형은 건설업.
                  ...            
8117      산업유형은 전문, 과학 및 기술 서비스업.
8118              산업유형은 금융 및 보험업.
8119              산업유형은 금융 및 보험업.
8120                   산업유형은 제조업.
8121             산업유형은 숙박 및 음식점업.
Name: bq1, Length: 8122, dtype: object

In [36]:
train4['bq2'] = train4['bq2'].map(bq2_to_text)

In [37]:
train4['bq2']

0                    근무업체 유형은 학교법인 혹은 학원.
1           근무업체 유형은 정부기관 및 산하기관(공기업 포함).
2       근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함).
3                    근무업체 유형은 학교법인 혹은 학원.
4       근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함).
                      ...                
8117        근무업체 유형은 정부기관 및 산하기관(공기업 포함).
8118    근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함).
8119    근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함).
8120    근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함).
8121    근무업체 유형은 민간회사 또는 개인사업체(외국인회사 포함).
Name: bq2, Length: 8122, dtype: object

In [38]:
train4['bq3'] = train4['bq3'].map(bq3_to_text)

In [39]:
train4['bq3']

0         근무업체 규모는 50명 ~ 99명.
1          근무업체 규모는 5명 ~ 49명.
2        근무업체 규모는 100명 ~299명.
3         근무업체 규모는 50명 ~ 99명.
4          근무업체 규모는 5명 ~ 49명.
                ...          
8117    근무업체 규모는 300명 ~ 999명.
8118       근무업체 규모는 5명 ~ 49명.
8119      근무업체 규모는 50명 ~ 99명.
8120       근무업체 규모는 5명 ~ 49명.
8121       근무업체 규모는 5명 ~ 49명.
Name: bq3, Length: 8122, dtype: object

In [40]:
train4['bq5_1'] = train4['bq5_1'].map(bq5_1_to_text)

In [41]:
train4['bq5_1']

0                훈련기간은 필요없다.
1        훈련기간은 6개월 이상~1년 미만.
2                훈련기간은 필요없다.
3       훈련기간은 1개월 이상~3개월 미만.
4                훈련기간은 필요없다.
                ...         
8117             훈련기간은 필요없다.
8118             훈련기간은 필요없다.
8119             훈련기간은 필요없다.
8120             훈련기간은 필요없다.
8121             훈련기간은 필요없다.
Name: bq5_1, Length: 8122, dtype: object

In [42]:
def bq6_to_text(num:int):
  if num == 1:
    return '관련된 업무 경험은 1개월 미만.'
  elif num == 2:
    return '관련된 업무 경험은 1개월 이상~3개월 미만.'
  elif num == 3:
    return '관련된 업무 경험은 3개월 이상~6개월 미만.'
  elif num == 4:
    return '관련된 업무 경험은 6개월 이상~1년 미만.'
  elif num == 5:
    return '관련된 업무 경험은 1년 이상~2년 미만.'
  elif num == 6:
    return '관련된 업무 경험은 2년 이상.'
  elif num == 7:
    return '관련된 업무 경험은 필요없음.'

train4['bq6'] = train4['bq6'].map(bq6_to_text)

In [44]:
train4['bq6']

0                관련된 업무 경험은 필요없음.
1        관련된 업무 경험은 6개월 이상~1년 미만.
2         관련된 업무 경험은 1년 이상~2년 미만.
3        관련된 업무 경험은 6개월 이상~1년 미만.
4                관련된 업무 경험은 필요없음.
                  ...            
8117    관련된 업무 경험은 3개월 이상~6개월 미만.
8118    관련된 업무 경험은 1개월 이상~3개월 미만.
8119             관련된 업무 경험은 필요없음.
8120            관련된 업무 경험은 2년 이상.
8121     관련된 업무 경험은 6개월 이상~1년 미만.
Name: bq6, Length: 8122, dtype: object

In [45]:
def bq7_to_text(num:int):
  if num == 1:
    return '업무 수행 교육수준은 중학교 졸업 이하.'
  elif num == 2:
    return '업무 수행 교육수준은 고등학교 졸업.'
  elif num == 3:
    return '업무 수행 교육수준은 고등학교 졸업 후 직업교육(6개월 이상) 이수.'
  elif num == 4:
    return '업무 수행 교육수준은 전문대학교 졸업.'
  elif num == 5:
    return '업무 수행 교육수준은 대학교 졸업.'
  elif num == 6:
    return '업무 수행 교육수준은 대학원 석사 졸업.'
  elif num == 7:
    return '업무 수행 교육수준은 대학원 박사 졸업.'

train4['bq7'] = train4['bq7'].map(bq7_to_text)

In [47]:
train4['bq7']

0          업무 수행 교육수준은 대학교 졸업.
1          업무 수행 교육수준은 대학교 졸업.
2          업무 수행 교육수준은 대학교 졸업.
3          업무 수행 교육수준은 대학교 졸업.
4          업무 수행 교육수준은 대학교 졸업.
                 ...          
8117    업무 수행 교육수준은 대학원 석사 졸업.
8118       업무 수행 교육수준은 대학교 졸업.
8119       업무 수행 교육수준은 대학교 졸업.
8120       업무 수행 교육수준은 대학교 졸업.
8121     업무 수행 교육수준은 전문대학교 졸업.
Name: bq7, Length: 8122, dtype: object

In [48]:
def bq24_to_text(num:int):
  if num == 1:
    return '나의 성별은 남성이다.'
  elif num == 2:
    return '나의 성별은 여성이다.'

train4['bq24'] = train4['bq24'].map(bq24_to_text)

In [49]:
train4['bq24']

0       나의 성별은 여성이다.
1       나의 성별은 남성이다.
2       나의 성별은 남성이다.
3       나의 성별은 여성이다.
4       나의 성별은 남성이다.
            ...     
8117    나의 성별은 남성이다.
8118    나의 성별은 여성이다.
8119    나의 성별은 남성이다.
8120    나의 성별은 여성이다.
8121    나의 성별은 남성이다.
Name: bq24, Length: 8122, dtype: object

In [50]:
def bq25_to_text(num:int):
  return '나의 나이는 ' + str(num) + '살이다.'

train4['bq25'] = train4['bq25'].map(bq25_to_text)


In [52]:
train4['bq25']

0       나의 나이는 42살이다.
1       나의 나이는 45살이다.
2       나의 나이는 38살이다.
3       나의 나이는 25살이다.
4       나의 나이는 49살이다.
            ...      
8117    나의 나이는 45살이다.
8118    나의 나이는 33살이다.
8119    나의 나이는 45살이다.
8120    나의 나이는 36살이다.
8121    나의 나이는 37살이다.
Name: bq25, Length: 8122, dtype: object

In [53]:
def bq26_to_text(num:int):
  if num == 1:
    return '나의 최종학력은 중학교 졸업 이하이다.'
  elif num == 2:
    return '나의 최종학력은 고등학교 졸업이다.'
  elif num == 3:
    return '나의 최종학력은 2~3년제 대학교 졸업이다.'
  elif num == 4:
    return '나의 최종학력은 대학교 졸업이다.'
  elif num == 5:
    return '나의 최종학력은 대학원 석사 졸업이다.'
  elif num == 6:
    return '나의 최종학력은 대학원 박사 졸업이다.'

train4['bq26'] = train4['bq26'].map(bq26_to_text)


In [55]:
train4['bq26']

0             나의 최종학력은 대학교 졸업이다.
1             나의 최종학력은 대학교 졸업이다.
2             나의 최종학력은 대학교 졸업이다.
3             나의 최종학력은 대학교 졸업이다.
4             나의 최종학력은 대학교 졸업이다.
                  ...           
8117       나의 최종학력은 대학원 박사 졸업이다.
8118          나의 최종학력은 대학교 졸업이다.
8119          나의 최종학력은 대학교 졸업이다.
8120       나의 최종학력은 대학원 석사 졸업이다.
8121    나의 최종학력은 2~3년제 대학교 졸업이다.
Name: bq26, Length: 8122, dtype: object

In [57]:
def bq28_to_text(num:int):
  try:
    num = int(num)
    if num == 1:
      return '나의 계약형태는 정규직이다.'
    elif num == 2:
      return '나의 계약형태는 비정규직이다.'
  except:
    return '나의 계약형태는 알 수 없다.'

train4['bq28'] = train4['bq28'].map(bq28_to_text)

In [58]:
train4['bq28']

0       나의 계약형태는 정규직이다.
1       나의 계약형태는 정규직이다.
2       나의 계약형태는 정규직이다.
3       나의 계약형태는 정규직이다.
4       나의 계약형태는 정규직이다.
             ...       
8117    나의 계약형태는 정규직이다.
8118    나의 계약형태는 정규직이다.
8119    나의 계약형태는 정규직이다.
8120    나의 계약형태는 정규직이다.
8121    나의 계약형태는 정규직이다.
Name: bq28, Length: 8122, dtype: object

In [59]:
def bq29_to_text(num:int):
  return '주당 근로시간은 ' + str(num) + '시간이다.'

train4['bq29'] = train4['bq29'].map(bq29_to_text)


In [61]:
train4['bq29']

0       주당 근로시간은 40시간이다.
1       주당 근로시간은 40시간이다.
2       주당 근로시간은 48시간이다.
3       주당 근로시간은 40시간이다.
4       주당 근로시간은 40시간이다.
              ...       
8117    주당 근로시간은 40시간이다.
8118    주당 근로시간은 45시간이다.
8119    주당 근로시간은 40시간이다.
8120    주당 근로시간은 40시간이다.
8121    주당 근로시간은 46시간이다.
Name: bq29, Length: 8122, dtype: object

In [62]:
def bq30_1_to_text(num:int):
  try:
    num = int(num)
    return '나의 근로소득은 ' + str(num) + '만원이다.'
  except:
    return '나의 근로소득은 알 수 없다.'

train4['bq30_1'] = train4['bq30_1'].map(bq30_1_to_text)

In [64]:
train4['bq30_1']

0       나의 근로소득은 4000만원이다.
1       나의 근로소득은 4000만원이다.
2       나의 근로소득은 4300만원이다.
3       나의 근로소득은 3500만원이다.
4       나의 근로소득은 5700만원이다.
               ...        
8117    나의 근로소득은 5100만원이다.
8118    나의 근로소득은 3100만원이다.
8119    나의 근로소득은 8000만원이다.
8120    나의 근로소득은 5700만원이다.
8121    나의 근로소득은 3700만원이다.
Name: bq30_1, Length: 8122, dtype: object

In [65]:
def bq30_2_to_text(num:int):
  try:
    num = int(num)
    return '나의 초임임금은 ' + str(num) + '만원이다.'
  except:
    return '나의 초임임금은 알 수 없다.'

train4['bq30_2'] = train4['bq30_2'].map(bq30_2_to_text)

In [68]:
train4['bq30_2']

0       나의 초임임금은 2600만원이다.
1       나의 초임임금은 3000만원이다.
2       나의 초임임금은 3000만원이다.
3       나의 초임임금은 3300만원이다.
4       나의 초임임금은 2700만원이다.
               ...        
8117    나의 초임임금은 4000만원이다.
8118    나의 초임임금은 2600만원이다.
8119    나의 초임임금은 3000만원이다.
8120    나의 초임임금은 3200만원이다.
8121    나의 초임임금은 2700만원이다.
Name: bq30_2, Length: 8122, dtype: object

In [69]:
def bq30_3_to_text(num:int):
  try:
    num = int(num)
    return '나의 순수입은 ' + str(num) + '만원이다.'
  except:
    return '나의 순수입은 알 수 없다.'

train4['bq30_3'] = train4['bq30_3'].map(bq30_3_to_text)

In [70]:
train4['bq30_3']

0       나의 순수입은 알 수 없다.
1       나의 순수입은 알 수 없다.
2       나의 순수입은 알 수 없다.
3       나의 순수입은 알 수 없다.
4       나의 순수입은 알 수 없다.
             ...       
8117    나의 순수입은 알 수 없다.
8118    나의 순수입은 알 수 없다.
8119    나의 순수입은 알 수 없다.
8120    나의 순수입은 알 수 없다.
8121    나의 순수입은 알 수 없다.
Name: bq30_3, Length: 8122, dtype: object